# Bose-Hubbard Model

In [ ]:
using KadanoffBaym
using LinearAlgebra
using NumericalIntegration
using Parameters
# using Plots

using PyPlot
using PyCall
using ProgressMeter
qt = pyimport("qutip")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## KadanoffBaym.jl

In [ ]:
# NOTE: not type-stable
NumericalIntegration._zero(x, y::Vector{<:AbstractMatrix}) = zero(first(y))

# Container for problem data
struct ProblemData
  GL::GreenFunction{ComplexF64, Array{ComplexF64,4}, Lesser}
  GG::GreenFunction{ComplexF64, Array{ComplexF64,4}, Greater}
  ΣL::GreenFunction{ComplexF64, Array{ComplexF64,4}, Lesser}
  ΣG::GreenFunction{ComplexF64, Array{ComplexF64,4}, Greater}
  ΣH::GreenFunction{ComplexF64, Array{ComplexF64,4}, Greater}
  h::Matrix{ComplexF64}
  U::Float64
  
  # Initialize problem
  function ProblemData(GL0::Matrix{ComplexF64}, h::Matrix{ComplexF64}, U::Float64, t0::Float64)
    @assert h == h' "A complex Hamiltonian requires revision of the equations"
    
    data = new(
      GreenFunction(reshape(GL0, size(GL0)...,1,1), Lesser),
      GreenFunction(reshape(GL0 - 1.0im * I, size(GL0)...,1,1), Greater),
      GreenFunction(zeros(ComplexF64, size(GL0)...,1,1), Lesser),
      GreenFunction(zeros(ComplexF64, size(GL0)...,1,1), Greater),
      GreenFunction(zeros(ComplexF64, size(GL0)...,1,1), Greater),
      h,
      U
    )
    
    return data
  end
end

# Vertical rhs
function rhs_vert(data, times, t1, t2)
  @unpack GL,GG,ΣL,ΣG,ΣH,h = data
  
  # real-time collision integral
  ∫dt(i,j,A,B) = sign(j-i) * integrate(times[min(i,j):max(i,j)], 
    [A[t1,t] * B[t,t2] for t=min(i,j):max(i,j)], TrapezoidalFast())
  
  dGL = -1.0im * ((h - ΣH[t1,t1]) * GL[t1,t2] + ∫dt(1,t1,ΣG,GL) - ∫dt(1,t1,ΣL,GL) + ∫dt(1,t2,ΣL,GL) - ∫dt(1,t2,ΣL,GG))
  dGG = -1.0im * ((h - ΣH[t1,t1]) * GG[t1,t2] + ∫dt(1,t1,ΣG,GG) - ∫dt(1,t1,ΣL,GG) + ∫dt(1,t2,ΣG,GL) - ∫dt(1,t2,ΣG,GG))
  return [dGL, dGG]
end

# Diagonal rhs
function rhs_diag(data, times, t1)
  @unpack GL,GG,h,ΣL,ΣG,ΣH = data
  
  # commutator
  ⋊(a,b) = a * b - b * a
  
  # real-time collision integral
  ∫dt(A,B) = integrate(times, [A[t1,t] * B[t,t1] for t=1:t1], TrapezoidalFast())
  I = ∫dt(ΣG,GL) - ∫dt(ΣL,GG) + ∫dt(GL,ΣG) - ∫dt(GG,ΣL)
  
  dGL = -1.0im * ((h - ΣH[t1,t1]) ⋊ GL[t1,t1] + I)
  dGG = -1.0im * ((h - ΣH[t1,t1]) ⋊ GG[t1,t1] + I)
  return [dGL, dGG]
end

# Self-energy
function calculate_Σ!(data, times, t1, t2)
  @unpack GL,GG,ΣL,ΣG,ΣH,h,U = data
  
  if (n = size(GL,3)) > size(ΣL,3)
    resize!(ΣL, n)
    resize!(ΣG, n)
    resize!(ΣH, n)
  end
  
  ∫dt(i,j,A) = sign(j-i) * integrate(times[min(i,j):max(i,j)], A, TrapezoidalFast())

  ΣL[t1,t2] = -(U/2)^2 * (16GL[t1,t2] .* GL[t1,t2] .* transpose(GG[t2,t1]))
  ΣG[t1,t2] = -(U/2)^2 * (16GG[t1,t2] .* GG[t1,t2] .* transpose(GL[t2,t1]))
#   ΣH[t1,t2] = (diagm ∘ diag)(
#     -(U/2) * (3GL[t1,t1] + GG[t1,t1])
#     -(U/2)^2 * 2 * (
#       ∫dt(1,t1,[(
#         + GG[t1,t] .* (12GL[t,t] + 4GG[t,t]) .* GL[t,t1]
#         - GL[t1,t] .* (12GL[t,t] + 4GG[t,t]) .* GG[t,t1]
#         + GG[t1,t] * (12GL[t,t] + 4GG[t,t]) * GL[t,t1] 
#         - GL[t1,t] * (12GL[t,t] + 4GG[t,t]) * GG[t,t1]) for t=1:t1])))
end

# Integration boundaries
tspan = (0.0, 4.0)

# Problem data
data = begin

  # Parameters
  h = ComplexF64[2.0 π * 0.1; π * 0.1 0.0];
  U = 0.2

  # Initial condition
  GL0 = -1.0im * [1e-5 0; 0 0]
  
  ProblemData(GL0, h, U, tspan[1])
end

# Integration
state, _ = kbsolve(
  (u, x...) -> rhs_vert(data, x...), 
  (u, x...) -> (println(" t: $(x[1][x[2]])"); rhs_diag(data, x...)), 
  [data.GL, data.GG], 
  tspan[1], 
  tspan[2]; 
  update_time! = (x...) -> calculate_Σ!(data, x...), 
  atol=1e-8, 
  rtol=1e-6, 
  init_dt=1e-4,
  max_dt=1e0)

In [ ]:
length(state.t)

## QuTiP benchmark

In [ ]:
J = data.h[2,1]
ω₁ = data.h[1,1]
ω₂ = data.h[2,2]
λ = 0.0

times = range(state.t[1], stop=state.t[end], length=length(state.t))
n = length(times) - 1
stop = Int(n/2) + 1

times_tau = 2 .* vcat([-times[stop - (k - 1)] for k in 1:stop - 1], times[1:stop]);

In [ ]:
# maximal number of photons
# Fock-space dimension will be n_max + 1
n_max = 7

# initial state
p = 1e-5
psi0_list = [sqrt(p)qt.basis(n_max + 1, 1) + sqrt(1 - p)qt.basis(n_max + 1, 0), qt.basis(n_max + 1, 0)]
# psi0_list = [qt.basis(n_max + 1, 1), qt.basis(n_max + 1, 0)]
psi0 = qt.tensor(psi0_list)

# define annihilation operators
a_1_list = [qt.destroy(n_max + 1), qt.qeye(n_max + 1)]
a_1  = qt.tensor(a_1_list)    
a_2_list = [qt.qeye(n_max + 1), qt.destroy(n_max + 1)]
a_2  = qt.tensor(a_2_list)    

# Hamiltonian
H  = J * a_1.dag() * a_2
H += H.dag();
H += ω₁ * a_1.dag() * a_1 + ω₂ * a_2.dag() * a_2; # do not give qutip a non-Hermitian Hamiltonian
# H += 0.5 * U * a_1.dag() * a_1 * a_1.dag() * a_1
# H += 0.5 * U * a_2.dag() * a_2 * a_2.dag() * a_2
H += 0.5 * U * a_1.dag() * a_1.dag() * a_1 * a_1
H += 0.5 * U * a_2.dag() * a_2.dag() * a_2 * a_2

observables = [a_1.dag()*a_1, a_2.dag()*a_2];

In [ ]:
# quickly solve once for observables
me = qt.mesolve(H, psi0, times, [sqrt(λ)a_1, sqrt(λ)a_2], observables)

# solve for the time-dependent density matrix
t_sols = qt.mesolve(H, psi0, times, [sqrt(λ)a_1, sqrt(λ)a_2]); # t_sols.states returns state vectors

In [ ]:
@assert n + 1 == length(t_sols.states)

tau_t_sols_11 = Dict()
tau_t_sols_22 = Dict()
@showprogress 1 for k in 1:(n + 1)
#     tau_t_sols_11[k] = qt.mesolve(H, a_1 * t_sols.states[k] * t_sols.states[k].dag(), times).states
#     tau_t_sols_22[k] = qt.mesolve(H, a_2 * t_sols.states[k] * t_sols.states[k].dag(), times).states
    tau_t_sols_11[k] = qt.mesolve(H, a_1 * t_sols.states[k], times).states
    tau_t_sols_22[k] = qt.mesolve(H, a_2 * t_sols.states[k], times).states    
end

a_1_dag_a_1 = zeros(ComplexF64, n + 1, n + 1)
a_2_dag_a_2 = zeros(ComplexF64, n + 1, n + 1)
@showprogress 1 for k in 1:(n + 1)
    for l in 1:(n + 1)
        a_1_dag_a_1[k, l] = (a_1.dag() * tau_t_sols_11[k][l]).tr()    
        a_2_dag_a_2[k, l] = (a_2.dag() * tau_t_sols_22[k][l]).tr()   
    end
end

In [ ]:
# reshape the above array to fit into our two-time "matrix" structure 
# see the plot below for illustration
unskewed_a_1_dag_a_1 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
unskewed_a_2_dag_a_2 = zeros(ComplexF64, n + 1, 2*(n + 1) - 1)
for (k, x) in enumerate([a_1_dag_a_1[k, :] for k in 1:(n + 1)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_1_dag_a_1[k, ind] = y 
    end
end

for (k, x) in enumerate([a_2_dag_a_2[k, :] for k in 1:(n + 1)])
    for (l, y) in enumerate(x)
        ind = k + l - 1 # verify the -1 relative to the original python code
        unskewed_a_2_dag_a_2[k, ind] = y  
    end
end

In [ ]:
figure(figsize=(6, 2))

subplot(121)
imshow(real(a_1_dag_a_1), cmap="plasma")

subplot(122)
imshow(real(unskewed_a_1_dag_a_1), cmap="plasma")

tight_layout()

## Plotting

In [ ]:
function equal_time_values(GX::Array{ComplexF64, 4})::Array{Array{ComplexF64, 1}, 2}
    
    dim = size(GX)[1]
    Y = [zeros(ComplexF64, n + 1) for _ in 1:dim, _ in 1:dim]
    for b in 1:dim, a in 1:dim
        Y[a, b] = [GX[a, b, k, k] for k in 1:n+1]
    end
    
    return Y
end 

GL_tt = equal_time_values(data.GL.data)
GG_tt = equal_time_values(data.GG.data);

In [ ]:
function full_tau_array(X::Array{ComplexF64, 2})::Array{ComplexF64, 1}
    Y = [X[stop - (k - 1), stop + (k - 1)] for k in 1:stop]
    Y_reversed = [Y[stop - (k - 1)] for k in 1:stop]
    return vcat(-conj(Y_reversed[1:end-1]), Y)    
end

function difference_time_values(GX::Array{Array{ComplexF64, 2}, 2})::Array{Array{ComplexF64, 1}, 2}
    dim = size(GX)[1]
    Y_full = [zeros(ComplexF64, n + 1) for _ in 1:dim, _ in 1:dim]
    for b in 1:dim, a in 1:dim
        Y_full[a, b] = full_tau_array(GX[a, b])
    end 
    
    return Y_full
end

In [ ]:
using Interpolations

GL_itp = interpolate((state.t, state.t), data.GL.data[1,1,:,:], Gridded(Linear()))
GG_itp = interpolate((state.t, state.t), data.GG.data[1,1,:,:], Gridded(Linear()));

In [ ]:
# # Interpolation
# pyinterpolate = pyimport("scipy.interpolate")

# const interp2d = pyinterpolate.interp2d
# # const rectSpline = interpolate.RectBivariateSpline

# # Data
# GL11 = [data.GL[1,1,t1,t2] for t1 ∈ eachindex(state.t), t2 ∈ eachindex(state.t)]

# # Grid
# time_grid = hcat([state.t for _ in eachindex(state.t)]...)
# tX = vcat([eachcol(time_grid)...]...)
# tY = vcat([eachrow(time_grid)...]...)

# GL_re_itp = interp2d(tX, tY, vcat([eachrow(real(GL11))...]...))
# GL_im_itp = interp2d(tX, tY, vcat([eachrow(imag(GL11))...]...))
# GL_itp_(t1,t2) = GL_re_itp(t1,t2) + 1.0im * GL_im_itp(t1,t2)

In [ ]:
# GL = hcat([GL_itp_(t2, times) for t2 in times]...)

GL = [GL_itp(t1, t2) for t1 in times, t2 in times]
# GG = [GG_itp(t1, t2) for t1 in times, t2 in times]

# GL_tau = full_tau_array(GL)
# GG_tau = full_tau_array(GG)

# a_1_dag_a_1_tau = imag(full_tau_array(1.0im .* unskewed_a_1_dag_a_1))
# a_2_dag_a_2_tau = imag(full_tau_array(1.0im .* unskewed_a_2_dag_a_2));

In [ ]:
plot(state.t, data.GL[1,1,:,:] |> imag |> diag)
plot!(times, GL[:,:] |> imag |> diag)

In [ ]:
T = state.t[end]

In [ ]:
figure(figsize=(12, 6))
subplot(221)
plot(times, me.expect[1], c="r", ls="--")
plot(times, me.expect[2], c="r", ls="--")
plot(state.t, -imag(GL_tt[1, 1]), c="b", alpha=0.5)
plot(state.t, -imag(GL_tt[2, 2]), c="g", alpha=0.5)
plot(state.t, -imag(GG_tt[1, 1]) .- 1, c="k", alpha=0.5)
plot(state.t, -imag(GG_tt[2, 2]) .- 1, c="k", alpha=0.5)
xlim(0, T)
# ylim(0, N_0)
xlabel("\$t\$")
plt.ticklabel_format(axis="y", style="sci", scilimits=(-2,2))

# subplot(222)
# plot(times_tau, a_1_dag_a_1_tau, c="r", ls="--")
# plot(times_tau, a_2_dag_a_2_tau, c="r", ls="--")
# plot(times_tau, -imag(GL_tau[1, 1]) + 0imag(GG_tau[1, 1]), c="b", alpha=0.5)
# plot(times_tau, -imag(GL_tau[2, 2]) + 0imag(GG_tau[2, 2]), c="g", alpha=0.5)
# xlim(-T, T)
# xlabel("\$\\tau\$")
# plt.ticklabel_format(axis="y", style="sci", scilimits=(-2,2))

tight_layout()

In [ ]:
plot(times_tau, -imag(GL_tau) - a_1_dag_a_1_tau)
plot(times_tau, -imag(GL_tau) - a_2_dag_a_2_tau)

In [ ]:
figure(figsize=(12, 3))
subplot(121)
plot(times, -imag(GL_tt[1, 1]) - me.expect[1], c="k", ls="--")
plot(times, -imag(GG_tt[2, 2]) .- 1 - me.expect[2], c="g", ls="-")
xlim(0, T)
xlabel("\$t\$")

subplot(122)
# plot(times_tau, -imag(GL_tau[1, 1]) - a_1_dag_a_1_tau, c="k", ls="--")
# plot(times_tau, -imag(GL_tau[2, 2]) - a_2_dag_a_2_tau, c="k", ls="--")
# axhline(0.0, c="r", ls=":")
xlim(-T, T)
xlabel("\$\\tau\$")
 
tight_layout()

In [ ]:
plot(times, [GL[1, 1][1, k] for k in 1:n+1])
plot(times, [a_1_dag_a_1[1, k] for k in 1:n+1])